# BigQuery - BQML - Export and Serving

Setup Parameters For Model Deployment

In [113]:
MODEL_NAME='BQML-DIGITS'
ENDPOINT_NAME='DIGITS'
MODEL_DIR='gs://statmike-models/digits/bqml'
FRAMEWORK='TENSORFLOW'
params = {"MODEL_DIR":MODEL_DIR}
REGION='us-central1'
CONTAINER_URI='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [85]:
%%bigquery --params $params
EXPORT MODEL `statmike-mlops.digits.digits_lr`
OPTIONS(URI = @MODEL_DIR)

""


This is a custom model.  We are importing it to AI Platform (Unified).  This is a good starting point:
- https://cloud.google.com/ai-platform-unified/docs/predictions/importing-custom-trained-model

Import the Model:
- https://cloud.google.com/ai-platform-unified/docs/predictions/importing-custom-trained-model#import_a_model_programmatically
- region of bucket and model must be the same

In [108]:
!gcloud beta ai models upload --region=$REGION --display-name=$MODEL_NAME --container-image-uri=$CONTAINER_URI --artifact-uri=$MODEL_DIR

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [5754222635709890560]...done.                            


In [110]:
!gcloud beta ai models list --region=$REGION --filter=display_name=$MODEL_NAME

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
---
artifactUri: gs://statmike-models/digits/bqml
containerSpec:
  imageUri: us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest
createTime: '2021-04-02T12:06:20.360877Z'
displayName: BQML-DIGITS
etag: AMEw9yM1kSWkg3E0aqT8GFxjdUKhmgMOteYFbx6AOYfnQ7VVi6rjIGItKTmN_sO0sio=
name: projects/691911073727/locations/us-central1/models/3796763184291905536
predictSchemata: {}
supportedDeploymentResourcesTypes:
- DEDICATED_RESOURCES
supportedExportFormats:
- exportableContents:
  - ARTIFACT
  id: custom-trained
supportedInputStorageFormats:
- jsonl
- csv
- tf-record
- tf-record-gzip
- file-list
supportedOutputStorageFormats:
- jsonl
updateTime: '2021-04-02T12:06:36.018533Z'


In [118]:
MODEL_ID=3796763184291905536

Deploy the Model:
    - https://cloud.google.com/ai-platform-unified/docs/predictions/deploy-model-api

In [114]:
!gcloud beta ai endpoints create --region=$REGION --display-name=$ENDPOINT_NAME

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [3011530462641258496]...done.                            
Created AI Platform endpoint: projects/691911073727/locations/us-central1/endpoints/9120088312588009472.


In [115]:
!gcloud beta ai endpoints list --region=$REGION --filter=display_name=$ENDPOINT_NAME

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
9120088312588009472  DIGITS


In [119]:
ENDPOINT_ID=9120088312588009472

In [120]:
!gcloud beta ai endpoints deploy-model $ENDPOINT_ID --region=$REGION --model=$MODEL_ID --display-name=$MODEL_NAME --traffic-split=0=100

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [2011731345365008384]...done.                            
Deployed a model to the endpoint 9120088312588009472. Id of the deployed model: 5470343027069288448.


In [121]:
DMODEL_ID=5470343027069288448

Get an online prediction:
- https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#online_predict_custom_trained-python

In [126]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`)
  )

In [139]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')
newob = {'instances':newob}
newob

{'instances': [{'p0': 0.0,
   'p1': 0.0,
   'p2': 0.0,
   'p3': 9.0,
   'p4': 14.0,
   'p5': 6.0,
   'p6': 0.0,
   'p7': 0.0,
   'p8': 0.0,
   'p9': 0.0,
   'p10': 10.0,
   'p11': 13.0,
   'p12': 4.0,
   'p13': 13.0,
   'p14': 2.0,
   'p15': 0.0,
   'p16': 0.0,
   'p17': 2.0,
   'p18': 14.0,
   'p19': 0.0,
   'p20': 0.0,
   'p21': 10.0,
   'p22': 6.0,
   'p23': 0.0,
   'p24': 0.0,
   'p25': 4.0,
   'p26': 9.0,
   'p27': 0.0,
   'p28': 0.0,
   'p29': 6.0,
   'p30': 8.0,
   'p31': 0.0,
   'p32': 0.0,
   'p33': 5.0,
   'p34': 8.0,
   'p35': 0.0,
   'p36': 0.0,
   'p37': 8.0,
   'p38': 7.0,
   'p39': 0.0,
   'p40': 0.0,
   'p41': 2.0,
   'p42': 11.0,
   'p43': 1.0,
   'p44': 0.0,
   'p45': 9.0,
   'p46': 5.0,
   'p47': 0.0,
   'p48': 0.0,
   'p49': 0.0,
   'p50': 6.0,
   'p51': 11.0,
   'p52': 4.0,
   'p53': 13.0,
   'p54': 3.0,
   'p55': 0.0,
   'p56': 0.0,
   'p57': 0.0,
   'p58': 1.0,
   'p59': 11.0,
   'p60': 16.0,
   'p61': 12.0,
   'p62': 0.0,
   'p63': 0.0}]}

In [141]:
import json
with open('newob.json','w') as f:
    json.dump(newob,f)

In [142]:
!gcloud beta ai endpoints predict $ENDPOINT_ID --region=$REGION --json-request=newob.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'predicted_target': ['0'], 'target_probs': [0.02363371530789796, 0.003446720128738627, 0.001599293545129464, 0.0007580118000224823, 2.437671616048318e-05, 0.001326708643650751, 0.0004530015313821015, 0.002578678260762049, 0.0009653040951390625, 0.965214189971117], 'target_values': ['6', '9', '4', '8', '1', '5', '3', '2', '7', '0']}]


In [148]:
import subprocess
prediction = subprocess.getoutput('gcloud beta ai endpoints predict '+str(ENDPOINT_ID)+' --region='+REGION+' --json-request=newob.json')

In [149]:
prediction

"Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]\n[{'predicted_target': ['0'], 'target_probs': [0.02363371530789796, 0.003446720128738627, 0.001599293545129464, 0.0007580118000224823, 2.437671616048318e-05, 0.001326708643650751, 0.0004530015313821015, 0.002578678260762049, 0.0009653040951390625, 0.965214189971117], 'target_values': ['6', '9', '4', '8', '1', '5', '3', '2', '7', '0']}]"

In [151]:
import re
import ast
pred = ast.literal_eval(re.search('({.+})',prediction).group(0))
pred

{'predicted_target': ['0'],
 'target_probs': [0.02363371530789796,
  0.003446720128738627,
  0.001599293545129464,
  0.0007580118000224823,
  2.437671616048318e-05,
  0.001326708643650751,
  0.0004530015313821015,
  0.002578678260762049,
  0.0009653040951390625,
  0.965214189971117],
 'target_values': ['6', '9', '4', '8', '1', '5', '3', '2', '7', '0']}

# Remove Resources
- undeploy-model
- remove endpoint
- remove model

In [160]:
!gcloud beta ai endpoints describe $ENDPOINT_ID --region=$REGION

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
createTime: '2021-04-02T12:10:45.394681Z'
displayName: DIGITS
etag: AMEw9yMcTPcNAhfyf1iF_g8oFT7vF4BBNt-yYKU0vgRSEXSbP4af79ZijhV-JctILMs=
name: projects/691911073727/locations/us-central1/endpoints/9120088312588009472
updateTime: '2021-04-02T13:08:43.180916Z'


In [153]:
!gcloud beta ai endpoints undeploy-model $ENDPOINT_ID --region=$REGION --deployed-model-id=$DMODEL_ID

Using endpoint [https://us-central1-aiplatform.googleapis.com/]


In [165]:
!echo "Y" | gcloud beta ai endpoints delete $ENDPOINT_ID --region=$REGION

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
This will delete endpoint [9120088312588009472]...

Do you want to continue (Y/n)?  


In [166]:
!gcloud beta ai models delete $MODEL_ID --region=$REGION

Using endpoint [https://us-central1-aiplatform.googleapis.com/]


In [167]:
!gsutil rm -r $MODEL_DIR

Removing gs://statmike-models/digits/bqml/#1617362257495261...
Removing gs://statmike-models/digits/bqml/saved_model.pb#1617362258134061...    
Removing gs://statmike-models/digits/bqml/variables/#1617362258019580...        
Removing gs://statmike-models/digits/bqml/variables/variables.data-00000-of-00001#1617362258266353...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://statmike-models/digits/bqml/variables/variables.index#1617362258370588...
/ [5 objects]                                                                   
Operation completed over 5 objects.                                              
